# Exercise 12: Dimensionality reduction

## Table of Contents

* Linear algebra based methods
    * PCA
    * jPCA
* Information theory based methods
    * Granger causality
    * ICA 
    * DCA
* Classifier
    * SVM

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as sps
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os

# For downloading data from the web
import urllib

# For retina displays only 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
%matplotlib inline

## Linear alegbra based methods 

### PCA

In [ ]:
# Scikit learn 
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge as RR
from sklearn.metrics import r2_score

#### PCA on test data 

#### PCA on neural data 

Due to some combination of simple experimental paradigms and more fundamental unknown principles of cortical processing, the neural data we recorded is often much lower dimensional than the recording dimensionality. This tutorial explores a few different ways of thinking about neural dimensionality. It contrasts DCA with the commonly used PCA.

**Run once to download the M1 reaching data** 

The file is 1.1 GB, so it may take a few minutes to download. This data is from the Sabes lab and is recordings from M1 while a monkey is reaching to different locations on a grid. This is the same data used in the DCA paper. In this tutorial, we won't cross validate the results as was done in the paper to keep things simple.

More information and data can be found here:

O'Doherty, Joseph E., Cardoso, Mariana M. B., Makin, Joseph G., & Sabes, Philip N. (2017). Nonhuman Primate Reaching with Multichannel Sensorimotor Cortex Electrophysiology [Data set]. Zenodo. http://doi.org/10.5281/zenodo.583331

In [ ]:
sabes_path = os.path.join('.', 'data', 'sabes-example-data.mat')

In [ ]:
if not os.path.isfile(sabes_path): # check if file was already downloaded
    urllib.request.urlretrieve('https://zenodo.org/record/583331/files/indy_20160627_01.mat?download=1', sabes_path)

In [ ]:
data = load_sabes_data(sabes_path, bin_width_s=.05, preprocess=True)

In [ ]:
keys = data.keys()
print(data.keys())
print(*[(key, data[key].shape) for key in keys])

In [ ]:
# M1 data
X = data['M1']
Xn = X / X.std(axis=0, keepdims=True) # normalized version will be used later

# Motor output 
Y = data['cursor']

In [ ]:
# Plotting the data and motor outplut
fig, (ax_neural, ax_cursor) = plt.subplots(nrows=2, figsize=(6,9), gridspec_kw={'height_ratios': [1,2]})

ax_neural.imshow(X[:1200].T, extent=[0, 1199*.05, 0, 108], cmap='gray_r', aspect='auto')
ax_neural.set_xlabel('Time (s)')
ax_neural.set_ylabel('Neuron')

ax_cursor.plot(*Y[:1200].T, c='k')
ax_cursor.set_xlabel('cursor x')
ax_cursor.set_ylabel('cursor y')

In [ ]:
# Running the PCA model for the data 
pca_model = PCA()
pca_model.fit(X)

In [ ]:
# Plotting the eigenvalues
eig_vals = pca_model.singular_values_
noise = 70 # would be estimated from multiple trials...
noise_cutoff = 4*noise/np.sqrt(3)

fig, ax = plt.subplots()
ax.plot(eig_vals)
ax.axhline(noise_cutoff, color='k', linestyle='dotted')
ax.set_ylim(0, eig_vals[0])
ax.set_ylabel("Eigenvalue")
ax.set_xlabel("Dimension #")

num_eigs_to_keep = np.nonzero(eig_vals > noise_cutoff)[0][-1] + 1
print(f"The top {num_eigs_to_keep} are above the noise floor.")

In [ ]:
# Plotting the first 4 PCAs
fig, axs = plt.subplots(nrows=num_eigs_to_keep)
for i in range(num_eigs_to_keep):
    axs[i].plot(pca_model.components_[i])
    axs[i].set_title(f"PCA component {i}")

In [ ]:
# Plotting the activity in the first 2 PCAs
fig, ax = plt.subplots()
X_pca_t = pca_model.transform(X)

PCA_1 = X_pca_t[:,0]
PCA_2 = X_pca_t[:,1]

ax.plot(PCA_1[:5], PCA_2[:5])

In [ ]:
# How much of the behavior do these components explain? 
max_dim = 30
lag = 4 # 200ms lag for the neural data for predicting behavior

In [ ]:
ds = np.arange(1, max_dim+1)
var = np.sum(pca_model.explained_variance_)

pca1_scores = np.zeros(ds.size)
for ii, d in enumerate(ds):
    Xd = pca_model.transform(X)[:, :d]
    rr_model = RR(alpha=1e-6)
    rr_model.fit(Xd[:-lag], Y[lag:])
    pca1_scores[ii] = r2_score(Y[lag:], rr_model.predict(Xd[:-lag]))
rr_model = RR(alpha=1e-6)
rr_model.fit(X[:-lag], Y[lag:])
max_score = r2_score(Y[lag:], rr_model.predict(X[:-lag]))

In [ ]:
fig, ax = plt.subplots()
ax.plot(ds, np.cumsum(pca_model.explained_variance_)[:ds.size] / var, label='Var. explained')
ax.plot(ds, pca1_scores / max_score, label=r'$R^2$')
ax.set_ylim(0, 1)
ax.legend(loc='best')
ax.set_xlabel('Projected dimensionality')
ax.set_ylabel('0-1 normalized metric')

### jPCA

In [ ]:
# Loading jPCA packages
import jPCA
from jPCA.util import load_churchland_data, plot_projections

**Run once to download the Churchland data**

Data and method from Churchland MM et al. (2012)  Structure of neural population dynamics during reaching. Nature. 

In [ ]:
churchland_path = os.path.join('.', 'data', 'churchland-example-data.mat')

In [ ]:
if not os.path.isfile(churchland_data): # check if file was already downloaded
    urllib.request.urlretrieve('https://www.dropbox.com/sh/2q3m5fqfscwf95j/AAC7fpAr5Sc3FN--nnZMX3sla/exampleData.mat?dl=1', churchland_path)

In [ ]:
datas, times = load_churchland_data(churchland_path)

In [ ]:
# Create a jPCA object
jpca = jPCA.JPCA(num_jpcs=2)
projected, full_data_var, pca_var_capt, jpca_var_capt = jpca.fit(datas, times=times, tstart=-50, tend=150)

In [ ]:
# Plot the projected data
plot_projections(projected)

## Information theory based methods

### ICA 

### Granger causality 

### DCA

In [ ]:
# Dynamical Component
from dca.data_util import load_sabes_data
from dca.dca import DynamicalComponentsAnalysis as DCA

In [ ]:
dca_model = DCA(T=10, d=109)
dca_model.estimate_data_statistics(X) # only need to estimate this once
dca_model.fit(X)

In [ ]:
# Plotting the activity in the first 2 PCAs
fig, ax = plt.subplots()
X_dca_t = dca_model.transform(X)

DCA_1 = X_dca_t[:,0]
DCA_2 = X_dca_t[:,1]

ax.plot(DCA_1[:10], DCA_2[:10])

In [ ]:
# Examining PI 
pi = np.zeros(ds.size)
dca_scores = np.zeros(ds.size)
max_pi = dca_model.score() # PI of data with no dimensionality reduction

In [ ]:
for ii, d in enumerate(ds):
    dca_model.fit_projection(d=d)
    pi[ii] = dca_model.score()
    Xd = dca_model.transform(X)
    rr_model = RR(alpha=1e-6)
    rr_model.fit(Xd[:-lag], Y[lag:])
    dca_scores[ii] = r2_score(Y[lag:], rr_model.predict(Xd[:-lag]))

In [ ]:
fig, ax = plt.subplots()
ax.plot(ds, pi / max_pi, label='PI')
ax.plot(ds, dca_scores / max_score, label=r'$R^2')
ax.ylim(0, 1)
ax.legend(loc='best')
ax.set_xlabel('Projected dimensionality')
ax.set_ylabel('0-1 normalized metric')